# <center> Process WhatsApp messages from NDOH

## Read in WhatsApp messages

- Currently saved as **`.txt`** files in **```data/doh_whatsapp```**

In [1]:
# TODO : find out how these files are saved/ stored ? - Is it manual ?

## Import Libraries

In [123]:
import pandas as pd
from pathlib import Path
import re

In [7]:
# read data
data_path = Path("../data/doh_whatsapp/")
data_files = list(data_path.glob("*.txt"))

In [74]:
len(data_files)

17

In [67]:
test_file = data_files[3]

In [68]:
test_file

PosixPath('../data/doh_whatsapp/2020-03-31-whatsapp-cases.txt')

In [124]:
## re

In [333]:
s1 = 'KWAZULU-NATAL: 12'
s2 = 'gauteng: 123'
s3 = 'Western Cape 433 '
s4 = 'Free state 85 '
s5 = 'NORTH WEST 11 '
s6 = 'MPUMALANGA 18 '
s7 = 'LIMPOPO 18 '
s8 = 'EASTERN CAPE 25'
s9 = 'NORTHERN CAPE 7'
s10 = 'UNALLOCATED, 19'

s = "\n".join([s1,s2,s3,s4,s5,s6,s7,s8,s9,s10])
s

'KWAZULU-NATAL: 12\ngauteng: 123\nWestern Cape 433 \nFree state 85 \nNORTH WEST 11 \nMPUMALANGA 18 \nLIMPOPO 18 \nEASTERN CAPE 25\nNORTHERN CAPE 7\nUNALLOCATED, 19'

In [350]:
kzn_query = r'(kwazulu[\s-]*natal|kzn)[\s:,]*(\d{1,5})'
gp_query = r'(gauteng)[\s:,]*(\d{1,5})'
wp_query = r'(Western Cape)[\s:,]*(\d{1,5})'
fs_query = r'(Free State)[\s:,]*(\d{1,5})'
nw_query = r'(North West)[\s:,]*(\d{1,5})'
mp_query = r'(Mpumalanga)[\s:,]*(\d{1,5})'
limp_query = r'(Limpopo)[\s:,]*(\d{1,5})'
ec_query = r'(Eastern Cape)[\s:,]*(\d{1,5})'
nc_query = r'(Northern Cape)[\s:,]*(\d{1,5})'
unk_query = r'(UNALLOCATED)[\s:,]*(\d{1,5})'

# other_queries = r"|".join([p for p in provinces])
# oq = f'{r"|".join([p for p in provinces])}[\s:]*[\d{1,5}]'
# oq

In [338]:
match = re.search(kzn_query, s, re.IGNORECASE)
if match:
    print('found', match.group())
else:
    print('did not find')

found KWAZULU-NATAL: 12


In [339]:
with open(data_file,'r') as reader :
    data = reader.readlines()
        
data_string = " ".join([d for d in data[8:20]])
print(data_string)
    


 GAUTENG: 31
 
 WESTERN CAPE: 16
 
 KWAZULU-NATAL: 12
 
 MPUMALANGA: 2
 
 LIMPOPO: 1
 
 At this stage, all cases were due to travel and no local transmissions have been confirmed. The NDOH are currently investigating 2 additional cases which may contain internal transmissions.



In [340]:
match = re.search(kzn_query, data_string, re.IGNORECASE)
if match:
    print('found', match.group())
else:
    print('did not find')

found KWAZULU-NATAL: 12


In [357]:
%%time
query_list = [kzn_query,
              gp_query, 
              wp_query,
              fs_query,
              nw_query, 
              mp_query,
              limp_query,
              ec_query,
              nc_query,
              unk_query]

for i,data_file in enumerate(data_files):
    print('[',i+1,']', data_file.stem)
    
    with open(data_file,'r') as reader :
        data = reader.readlines()
        
    data_string = " ".join([d for d in data[8:20]])
#     print(data_string)
#     print("********"*15)
    for i,q in enumerate(query_list):
        match = re.search(q, data_string, re.IGNORECASE)
        if match:
            print(f"query {i+1} of 10 : {match.group().strip()}")
        else:
            print(f"no match for query {i+1} of 10 ")
            
    print("--------"*10) 
    

[ 1 ] 2020-04-02-whatsapp-cases
query 1 of 10 : KwaZulu Natal: 206
query 2 of 10 : Gauteng: 663
query 3 of 10 : Western Cape: 353
query 4 of 10 : Free State: 84
query 5 of 10 : North West: 9
query 6 of 10 : Mpumalanga: 13
query 7 of 10 : Limpopo: 16
query 8 of 10 : Eastern Cape: 17
query 9 of 10 : Northern Cape: 7
query 10 of 10 : Unallocated: 94
--------------------------------------------------------------------------------
[ 2 ] 2020-03-23-whatsapp-cases
query 1 of 10 : KwaZulu-Natal 60
query 2 of 10 : Gauteng 207
query 3 of 10 : Western Cape 100
query 4 of 10 : Free State 13
query 5 of 10 : North West 4
query 6 of 10 : Mpumalanga 9
query 7 of 10 : Limpopo 4
query 8 of 10 : Eastern Cape 2
query 9 of 10 : Northern Cape 2
no match for query 10 of 10 
--------------------------------------------------------------------------------
[ 3 ] 2020-04-03-whatsapp-cases
query 1 of 10 : KwaZulu Natal: 215
query 2 of 10 : Gauteng: 672
query 3 of 10 : Western Cape: 374
query 4 of 10 : Free State:

In [69]:
with open(test_file,'r') as reader :
    data = reader.readlines()

In [70]:
data

['Current Status of Cases of COVID-19 in South Africa\n',
 '31 MARCH 2020 - 22:10\n',
 '\n',
 'Total cases: 1353\n',
 '31 Full recoveries (Confirmed Negative)  \n',
 '5 Deaths\n',
 '\n',
 'The breakdown per province of total infections is as follows(To be updated):\n',
 '633 Gauteng \n',
 '325  Western Cape\n',
 '179 KwaZulu Natal \n',
 '74 Free State  \n',
 '8 North West\n',
 '12 Mpumalanga\n',
 '14 Limpopo\n',
 '6 Northern Cape \n',
 '12 Eastern Cape \n',
 '90 Unknown\n',
 '\n',
 'The 14 health workers, including 3 specialist doctors, who were in contact with the COVID-19 patient who died are now in quarantine and being monitored.\n',
 '\n',
 'We express our condolences to the family and we thank the doctors and health workers who looked after the deceased patient until his time of passing.\n',
 '\n',
 '--\n',
 'For the latest news go to https://twitter.com/HealthZA or https://sacoronavirus.co.za/category/press-releases-and-notices/ \n',
 '\n',
 '📌Or reply with MENU to return to the 

In [119]:
provinces = ["Gauteng",
             "Western Cape",
             "KwaZulu Natal",
             "Free State",
             "North West",
             "Mpumalanga",
             "Limpopo",
             "Northern Cape",
             "Eastern Cape",
             "Unknown",
             "Unallocated"]

In [120]:
# remove newline characters and lower case data
proc_data = [d.title().strip("\n")for d in data]

In [122]:
%%time
for i,data_file in enumerate(data_files):
    
    with open(data_file,'r') as reader :
        data = reader.readlines()
    print(i+1, data_file.stem)
    print()
    proc_data = [d.title().strip("\n").replace("-"," ")for d in data]
    for prov in provinces:
        for d in proc_data:
            if prov.title() in d:
                print(d)
    print("--------"*10)
    

1 2020-04-02-whatsapp-cases

Gauteng: 663
Western Cape: 353
Kwazulu Natal: 206
Free State: 84
North West: 9
Mpumalanga: 13
Limpopo: 16
Northern Cape: 7
Eastern Cape: 17
Unallocated: 94
--------------------------------------------------------------------------------
2 2020-03-23-whatsapp-cases

Gauteng 207
Western Cape 100
Kwazulu Natal 60
Free State 13
North West 4
Mpumalanga 9
Limpopo 4
Northern Cape 2
Eastern Cape 2
Unknown 1
--------------------------------------------------------------------------------
3 2020-04-03-whatsapp-cases

Gauteng: 672
Western Cape: 374
Kwazulu Natal: 215
Free State: 84
North West: 9
Mpumalanga: 13
Limpopo: 16
Northern Cape: 7
Eastern Cape: 21
Unallocated: 94
--------------------------------------------------------------------------------
4 2020-03-31-whatsapp-cases

633 Gauteng 
325  Western Cape
179 Kwazulu Natal 
74 Free State  
8 North West
12 Mpumalanga
14 Limpopo
6 Northern Cape 
12 Eastern Cape 
90 Unknown
-------------------------------------------

In [71]:
heading = data[0]
date = data[1]
summary = data[2:8]
stats_breakdown = data[9:20]
end_text = data[21:]

In [72]:
print(f"heading : {heading}")
print("---"*25)

print(f"date : {date}")
print("---"*25)

print(f"summary : {[print(s) for s in summary]}")
print("---"*25)

print(f"stats_breakdown : {[print(stats) for stats in stats_breakdown]}")
print("---"*25)

print(f"end_text : {end_text}")
print("---"*25)


heading : Current Status of Cases of COVID-19 in South Africa

---------------------------------------------------------------------------
date : 31 MARCH 2020 - 22:10

---------------------------------------------------------------------------


Total cases: 1353

31 Full recoveries (Confirmed Negative)  

5 Deaths



The breakdown per province of total infections is as follows(To be updated):

summary : [None, None, None, None, None, None]
---------------------------------------------------------------------------
325  Western Cape

179 KwaZulu Natal 

74 Free State  

8 North West

12 Mpumalanga

14 Limpopo

6 Northern Cape 

12 Eastern Cape 

90 Unknown



The 14 health workers, including 3 specialist doctors, who were in contact with the COVID-19 patient who died are now in quarantine and being monitored.

stats_breakdown : [None, None, None, None, None, None, None, None, None, None, None]
---------------------------------------------------------------------------
end_text : ['We 

In [73]:
date.strip().split('-')

['31 MARCH 2020 ', ' 22:10']